In [6]:
%pip install yfinance

  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139051 sha256=16470bdd72f92c2c074edc44567af8f739d9353f27ad8616ef748613963e451c
  Stored in directory: /Users/jackcampbell/Library/Caches/pip/wheels/43/ef/2d/2c51d496bf084945ffdf838b4cc8767b8ba1cc20eb41588831
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import json

# Define available stock symbols
symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]


def fetch_stock_data(symbol):
    """Fetch historical stock data and compute daily log returns."""
    data = yf.download(symbol, period="1y")
    data["Log Return"] = np.log(data["Close"] / data["Close"].shift(1))
    return data["Log Return"].dropna()


# Pre-fetch data for all stocks
preloaded_data = {}

for symbol in symbols:
    log_returns = fetch_stock_data(symbol)
    mu, sigma = log_returns.mean(), log_returns.std()

    # Generate Normal distribution data
    x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
    y = (1 / (sigma * np.sqrt(2 * np.pi))) * \
        np.exp(-0.5 * ((x - mu) / sigma) ** 2)

    # Create Plotly figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode="lines",
                  name="Normal Distribution"))
    fig.add_trace(go.Histogram(
        x=log_returns, histnorm="probability density", opacity=0.5, name="Log Returns"))
    fig.update_layout(
        title=f"Normal Distribution of {symbol} Daily Log Returns",
        xaxis_title="Log Return",
        yaxis_title="Density",
        template="plotly_white"
    )

    # Store JSON representation of the plot
    preloaded_data[symbol] = fig.to_json()

# Convert preloaded data to JSON
preloaded_json = json.dumps(preloaded_data)

# Generate the standalone HTML file
html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Stock Log Returns</title>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>

  <h2>Select a Stock:</h2>
  <select id="stock-selector">
    {''.join(f'<option value="{symbol}">{symbol}</option>' for symbol in symbols)}
  </select>

  <div id="plot"></div>

  <script>
    // Preloaded plot data for all stocks
    var preloadedData = {preloaded_json};

    // Default stock selection
    var defaultSymbol = "{symbols[0]}";
    var plotData = JSON.parse(preloadedData[defaultSymbol]);

    // Render the default chart
    Plotly.newPlot('plot', plotData.data, plotData.layout);

    // Update chart when stock is selected
    document.getElementById("stock-selector").addEventListener("change", function() {{
      var symbol = this.value;
      var newPlotData = JSON.parse(preloadedData[symbol]);
      Plotly.react('plot', newPlotData.data, newPlotData.layout);
    }});
  </script>

</body>
</html>
"""

# Save the HTML file
file_name = "../plotly/distribution.html"
with open(file_name, "w") as f:
    f.write(html_template)

print(f"Standalone interactive HTML saved as {file_name}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Standalone interactive HTML saved as ../plotly/distribution.html
